In [ ]:
import os
import json
import pandas as pd

In [ ]:
# reading in the answers

df_answers = pd.read_csv('reddit_answers_big.csv', sep = ";")
df_answers.head(3)

In [ ]:
# grabbing the top answers

df_top_votes = df_answers.groupby('q_id')['votes'].idxmax()
df_top_answers = df_answers.loc[df_top_votes]

df_top_answers.rename(columns = {'text': 'answer'}, inplace = True)
df_top_answers.rename(columns = {'q_id': 'id'}, inplace = True)
df_top_answers.rename(columns = {'votes': 'answer_votes'}, inplace = True)

df_top_answers.head(3)

In [ ]:
# reading in the questions

df_questions = pd.read_csv('reddit_questions_big.csv', sep = ";")

# renaming columns
df_questions.rename(columns = {'text': 'question'}, inplace = True)
df_questions.rename(columns = {'votes': 'question_votes'}, inplace = True)

df_questions.head(3)

In [ ]:
# joining questions and answers on id

merged_df = df_questions.merge(df_top_answers, on = 'id')

merged_df.head(3)

In [ ]:
# cleaning data

merged_df.drop(['timestamp', 'date', 'Unnamed: 0'], axis = 1, inplace = True)

merged_df = merged_df.reindex(columns = ['id', 'question', 'answer', 'question_votes' 'answer_votes'])

merged_df.head(3)

In [ ]:
merged_df.info()

In [ ]:
# downsizing the data
merged_df = merged_df.sort_values(by = 'answer_votes', ascending = False)
merged_df_1k = merged_df[:1000]
merged_df_1k.head(3)

In [ ]:
# fine tuning format
questions, answers = merged_df_1k['question'], merged_df_1k['answer']

In [ ]:
qa_openai_format = [{'messages': [{"role": "system", "content": "Marv is a factual chatbot and reddit expert who likes to answer with bullets"},
                                  {"role": "user", "content": q},
                                  {"role": "assistant", "content": a}]} for q, a in zip(questions, answers)]

qa_openai_format[5]

In [ ]:
with open("training_data.jsonl", "w") as f:
    for entry in qa_openai_format:
        f.write(json.dumps(entry))
        f.write("\n")

In [ ]:
from collections import defaultdict

data_path = "training_data.jsonl"

# load the dataset
with open(data_path, 'r', encoding = 'utf-8') as f:
    dataset = [json.loads(line) for line in f]


# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

In [ ]:
# pip install openai
import os
import openai

# os.environ["OPENAI_API_KEY"] = "YOUR KEY GOES HERE"
# openai.api_key=os.getenv('OPENAI_API_KEY')

client = OpenAI()

In [ ]:
# time to finally start fine tuning (sending the training data to OpenAI)

client.files.create(
    file = open("training_data.jsonl", 'rb'),
    purpose = 'fine-tune'
)

In [ ]:
client.fine_tuning.jobs.create(
    training_file = 'file id outputted from last code block',
    model = 'gpt-3.5-turbo'
)

In [ ]:
client.fine_tuning.jobs.list()

In [ ]:
# after training

system_prompt = "Marv is a factual chatbot and reddit expert who likes to answer with bullets"
user_question = "Give me the dumbest thing you've ever done"

In [ ]:
response = client.chat.completions.creatte(
    model = "trained model (ft:gpt-3.5)",
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question}
    ]
)

print(response.choices[0].message.content)